In [1]:
import cv2
import os
import pandas as pd
import numpy as np

In [2]:
class AutoSegmentROI:
    def __init__(self):
        self.roi_pts = []
    
    def select_roi(self, image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Apply Otsu's thresholding
        ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        # Find contours in the thresholded image
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # Check if any contours were found
        if len(contours) == 0:
            print("No contours found in image")
            return None
        # Select the largest contour
        max_contour = max(contours, key=cv2.contourArea)
        # Get the bounding box of the contour
        x, y, w, h = cv2.boundingRect(max_contour)
        # Save the ROI points
        self.roi_pts = [(x, y), (x + w, y), (x + w, y + h), (x, y + h)]
        return self.roi_pts

In [5]:
# Path to input folder
input_folder = "D:\\master_thesis\\datasets\\histogram_equilization\\normal"
# Path to output folder
output_folder = "D:\\master_thesis\\datasets\\segmented\\normal"

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Create AutoSegmentROI object
segment_roi = AutoSegmentROI()

# Process each image in the input folder
for filename in os.listdir(input_folder):
    # Load image
    image_path = os.path.join(input_folder, filename)
    image = cv2.imread(image_path)
    # Get ROI points
    roi_pts = segment_roi.select_roi(image)
    # Extract ROI
    if roi_pts is not None:
        roi = cv2.fillPoly(np.zeros_like(image), [np.array(roi_pts)], (255, 255, 255))
        roi = cv2.bitwise_and(image, roi)
        # Save output image
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, roi)


In [ ]:
# Manual Segmentation using region of interest (ROI)
class segment_roi:
    def __init__(self):
        self.image = None
        self.roi_pts = []
        self.completed = False

#to select the region of interest 
    def select_roi(self, image):
        self.image = image
        clone = self.image.copy()
        cv2.namedWindow("Select ROI")
        cv2.setMouseCallback("Select ROI", self._select_roi_callback)

        while True:
            cv2.imshow("Select ROI", self._draw_roi(clone))
            key = cv2.waitKey(1) & 0xFF
            if key == ord("r"):
                self.roi_pts = []
                clone = self.image.copy()
            elif key == ord("c"):
                if len(self.roi_pts) == 4:
                    self.completed = True
                    cv2.destroyWindow("Select ROI")
                    return self.roi_pts
                else:
                    print ("Select four points to define ROI:")
            elif key ==27:
                cv2.destroyWindow("Select ROI")
                break
        return None
    
    def _select_roi_callback(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            if len(self.roi_pts) < 4:
                self.roi_pts.append((x,y))

#to draw the region of interest points required 
    def _draw_roi(self, image):
        for pt in self.roi_pts:
            cv2.circle(image, pt, 5, (0, 255, 0),2)

        if len(self.roi_pts) == 4:
            cv2.line(image, self.roi_pts[0], self.roi_pts[1], (0,255,0),2)
            cv2.line(image, self.roi_pts[1], self.roi_pts[2], (0,255,0),2)  
            cv2.line(image, self.roi_pts[2], self.roi_pts[3], (0,255,0),2)  
            cv2.line(image, self.roi_pts[3], self.roi_pts[0], (0,255,0),2)  
        return image         


In [ ]:
#input and output folder path
input_folder = "D:\master_thesis\datasets\histogram_equilization\glioma"
output_folder = "D:\master_thesis\datasets\segmented\glioma"


# Create output directory if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

#loop over all the images in the input folder
for filename in os.listdir(input_folder):
    #get filename from the filepath
    image_path = os.path.join(input_folder, filename)
    #load images
    image = cv2.imread(image_path)

    #create roi_selector object
    roi_selector = segment_roi()
    #to select the roi
    roi_pts = roi_selector.select_roi(image)
    
    

#to extract roi
    if roi_pts is not None:
        mask = np.zeros(image.shape[:2], dtype=np.unit8)
        roi_corners = np.array(roi_pts,dtype=np.int32)
        cv2.fillPoly(mask, [roi_corners], {255,255,255})
        roi = cv2.bitwise_and(image, image, mask=mask)

        #save segmented image to ouput folder with different filename
        output_filename = "segmented_" + filename
        output_path = os.path.join(output_folder, output_filename)
        cv2.imwrite(output_path, roi)
        cv2.imshow("ROI", roi)
        cv2.waitKey(0)


KeyboardInterrupt: 